# GroupfMRI17a - Control analyses - run using the least similar hidden area regions
## Using 5, 10, 15, and 20 regions
## Task context regions are a single set; including regions that decode 12 task rules
#### Analysis 1 - Full model, SRActFlow decoding
#### Control 1 - Remove nonlinearities
#### Run a standard permutation test, shuffling labels on predicted activation patterns to decode held-out activations


Takuya Ito

10/24/2019

In [33]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools_group_rsa_python3 as tools_group
import nibabel as nib
import EmpiricalSRActFlow_ANN_RSA_v3 as esr
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"
import pandas as pd
import time
from importlib import reload


In [34]:
cabn_l = np.squeeze(nib.load('/projects3/AnalysisTools/ColeAnticevicNetPartition/SeparateHemispheres/Q1-Q6_RelatedParcellation210.L.CorticalAreas_dil_Colors.32k_fs_LR.dlabel.nii').get_data())
cabn_r = np.squeeze(nib.load('/projects3/AnalysisTools/ColeAnticevicNetPartition/SeparateHemispheres/Q1-Q6_RelatedParcellation210.R.CorticalAreas_dil_Colors.32k_fs_LR.dlabel.nii').get_data())+180
cabn = np.hstack((cabn_l,cabn_r))

In [35]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
# networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
# Swap so that it's R->L
networkdef = np.loadtxt('/projects3/AnalysisTools/ColeAnticevicNetPartition/cortex_parcel_network_assignments.txt')
networkdef = np.hstack((networkdef[180:],networkdef[:180]))
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects3/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(list(xticks.keys()))
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# Load real motor response data

In [36]:
tools_group = reload(tools_group)
nResponses = 2
data_task_rh = np.zeros((len(glasser2),nResponses,len(subjNums)))
data_task_lh = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task_rh[:,:,scount] = np.real(tools_group.loadMotorResponses(subj,hand='Right'))
    data_task_lh[:,:,scount] = np.real(tools_group.loadMotorResponses(subj,hand='Left'))
    scount += 1

# -5 Hidden layers

#### Define wrapper function for SRActFlow

In [44]:
esr = reload(esr)
n_hiddenregions = -5
####################################################################################################################
#### Identify target vertices (in motor cortex)
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))

target_ind = []
for roi in targetROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)
####################################################################################################################
#### Generate trials to simulate empirical brain computational models
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_15stims_v2.csv' # Good
filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_15stims_v3.csv' # Great
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_6stims_v1.csv' # Great
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_50stims_v1.csv' # Great
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/tmp.csv'
# esr.constructTasks(n_stims=6,filename=filename)
trial_metadata = pd.read_csv(filename)
####################################################################################################################
##### Load FC mappings
tools_group = reload(tools_group)
global fc_input2hidden
global fc_12rule2hidden
inputtypes = ['color','ori','pitch','constant']
inputkeys = ['RED','VERTICAL','HIGH','CONSTANT']
fc_input2hidden = {}
eig_input2hidden = {}
i = 0
for inputtype in inputtypes:
    fc_input2hidden[inputkeys[i]], eig_input2hidden[inputkeys[i]] = tools_group.loadGroupActFlowFC(inputtype, n_hiddenregions=n_hiddenregions, pc_space=True)
    print(fc_input2hidden[inputkeys[i]].shape)
    i += 1

fc_12rule2hidden, eig_12rule2hidden = tools_group.loadGroupActFlowFC('12', n_hiddenregions=n_hiddenregions, pc_space=True)
print(fc_12rule2hidden.shape)
# Load hidden to motor resp mappings
fc_hidden2motorresp, eig_hidden2motorresp = tools_group.loadGroupActFlowFC('hidden2out', n_hiddenregions=n_hiddenregions, pc_space=True)
print(fc_hidden2motorresp.shape)
####################################################################################################################

def subjSRActFlow_12Rule_PCFC(subj,trial_metadata,n_hiddenregions):
    print('Subject ' + subj + '... Simulating ' + str(len(trial_metadata)) + ' Trials')
    obj = esr.EmpiricalActFlow(subj)
    # Input
    obj.fc_input2hidden = fc_input2hidden
    obj.eig_input2hidden = eig_input2hidden
    # Rules
    obj.fc_12rule2hidden = fc_12rule2hidden
    obj.eig_12rule2hidden = eig_12rule2hidden
    # hidden 2 motor
    obj.fc_hidden2motorresp = fc_hidden2motorresp
    obj.eig_hidden2motorresp = eig_hidden2motorresp
    
    obj.extractAllActivations(trial_metadata)

    actflow = obj.generateActFlowPredictions_12Rule_PCFC(thresh=0,n_hiddenregions=n_hiddenregions,verbose=False)
    actflow_control = obj.generateActFlowPredictions_12Rule_PCFC(thresh=None,n_hiddenregions=n_hiddenregions,verbose=False)
    del obj
    return actflow, actflow_control

inputs = []
for i in range(len(subjNums)):
    inputs.append((subjNums[i],trial_metadata,n_hiddenregions))

timestart = time.time()
pool = mp.Pool(processes=8)
results = pool.starmap_async(subjSRActFlow_12Rule_PCFC,inputs).get()
pool.close()
pool.join()
timeend = time.time()
print("time elapsed:", timeend-timestart)

actflow_predictions = np.zeros((len(subjNums),len(target_ind),4))
actflow_predictions_control = np.zeros((len(subjNums),len(target_ind),4))
scount = 0
for result in results:
    actflow_predictions[scount,:,:] = result[0]
    actflow_predictions_control[scount,:,:] = result[1]
    scount += 1



(500, 381)
(500, 381)
(500, 381)
(500, 381)
(500, 381)
(380, 8642)
Subject 013... Simulating 960 Trials
Subject 023... Simulating 960 Trials
Subject 017... Simulating 960 Trials
Subject 027... Simulating 960 Trials
Subject 031... Simulating 960 Trials
Subject 038... Simulating 960 Trials
Subject 041... Simulating 960 Trials
Subject 034... Simulating 960 Trials
Subject 035... Simulating 960 Trials
Subject 042... Simulating 960 Trials
Subject 032... Simulating 960 Trials
Subject 014... Simulating 960 Trials
Subject 028... Simulating 960 Trials
Subject 039... Simulating 960 Trials
Subject 024... Simulating 960 Trials
Subject 018... Simulating 960 Trials
Subject 037... Simulating 960 Trials
Subject 016... Simulating 960 Trials
Subject 021... Simulating 960 Trials
Subject 043... Simulating 960 Trials
Subject 033... Simulating 960 Trials
Subject 030... Simulating 960 Trials
Subject 040... Simulating 960 Trials
Subject 026... Simulating 960 Trials
Subject 045... Simulating 960 Trials
Subject 

#### Compute average activity for each response, for each subject

In [45]:
## No threshold
scount = 0
actflow_rh = np.zeros(data_task_rh.shape)
actflow_lh = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh[target_ind,0,scount] = actflow_predictions[scount,:,2]
    # RIND
    actflow_rh[target_ind,1,scount] = actflow_predictions[scount,:,3]
    # LMID
    actflow_lh[target_ind,0,scount] = actflow_predictions[scount,:,0]
    # LIND
    actflow_lh[target_ind,1,scount] = actflow_predictions[scount,:,1]


## -5 HIDDEN - Run across subject decoding on right-hand motor responses

In [46]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputRH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]

# rois = np.asarray([7,8,52])
# rois = np.asarray([8])
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind
#
realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh[roi_ind,:,:].copy()

#############################################################################################################
#### Run decoding
distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch, confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True, featsel=False,confusion=True,
                                                          ncvs=ncvs, nproc=nproc)

#############################################################################################################
#### Run statistics
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print('Activity flow accuracy =', statistics_rh[0,0])
print('p =', statistics_rh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))

Activity flow accuracy = 0.3020833333333333
p = 0.9999999791047559
Matched spatial correlation: -0.02935091570248489
Mismatched spatial correlation: 0.029350915702484798


## 5 HIDDEN - Run across subject decoding on left-hand motor responses

In [47]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputLH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]

rois = np.asarray([187, 188, 231])
rois = np.asarray([188])
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind

realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh[roi_ind,:,:].copy()


#############################################################################################################
#### Run decoding
distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch, confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,featsel=False,confusion=True,
                                                          ncvs=ncvs, nproc=nproc)
    

#############################################################################################################
#### Run statistics
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print('Activity flow accuracy =', statistics_lh[0,0])
print('p =', statistics_lh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))

Activity flow accuracy = 0.4375
p = 0.9516582343118908
Matched spatial correlation: -0.013052721829265269
Mismatched spatial correlation: 0.013052721829265994


___

## Run analysis for Control (No threshold) variant

#### Compute average activity for each response, for each subject

In [48]:
scount = 0
actflow_rh_ctrl = np.zeros(data_task_rh.shape)
actflow_lh_ctrl = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh_ctrl[target_ind,0,scount] = actflow_predictions_control[scount,:,2]
    # RIND
    actflow_rh_ctrl[target_ind,1,scount] = actflow_predictions_control[scount,:,3]
    # LMID
    actflow_lh_ctrl[target_ind,0,scount] = actflow_predictions_control[scount,:,0]
    # LIND
    actflow_lh_ctrl[target_ind,1,scount] = actflow_predictions_control[scount,:,1]


## -5 HIDDEN - (NO THRESHOLD CONTROL): Run across subject decoding on right-hand motor responses

In [49]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputRH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]


# rois = np.asarray([8,52,9])-1
# rois = np.asarray([8,52])-1
# rois = np.where(networkdef==networkmappings['smn'])[0]
# rois = np.asarray([7,8,52])
rois = np.asarray([8])
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind

realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh_ctrl[roi_ind,:,:].copy()

#############################################################################################################
#### Run decoding
distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch, confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,confusion=True,featsel=False,
                                                          ncvs=ncvs, nproc=nproc)

#############################################################################################################
#### Run decoding
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print('Activity flow accuracy =', statistics_rh[0,0])
print('p =', statistics_rh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))



Activity flow accuracy = 0.53125
p = 0.2136772946983039
Matched spatial correlation: -0.005211427699854849
Mismatched spatial correlation: -0.007806089928859505


## -5 HIDDEN - (NO THRESHOLD CONTROL): Run across subject decoding on left-hand motor responses

In [50]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputLH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]
# rois = np.asarray([188,189,232]) - 1
# rois = np.asarray([188,232]) - 1
# rois = np.asarray([189]) - 1
# rois = np.where(networkdef==networkmappings['smn'])[0]
# rois = np.asarray([187,188,231])
rois = np.asarray([188])
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind

realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh_ctrl[roi_ind,:,:].copy()


#############################################################################################################
#### Run decoding
distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch,confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,confusion=True,featsel=False,
                                                          ncvs=ncvs, nproc=nproc)
    

#############################################################################################################
#### Run statistics
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print('Activity flow accuracy =', statistics_lh[0,0])
print('p =', statistics_lh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))



Activity flow accuracy = 0.578125
p = 0.018041701146465264
Matched spatial correlation: 0.021502617359161553
Mismatched spatial correlation: 0.007813652349170402


___

# -10 Hidden layers

#### Define wrapper function for SRActFlow

In [72]:
esr = reload(esr)
n_hiddenregions = -10
####################################################################################################################
#### Identify target vertices (in motor cortex)
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))

target_ind = []
for roi in targetROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)
####################################################################################################################
#### Generate trials to simulate empirical brain computational models
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_15stims_v2.csv' # Good
filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_15stims_v3.csv' # Great
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_6stims_v1.csv' # Great
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_50stims_v1.csv' # Great
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/tmp.csv'
# esr.constructTasks(n_stims=6,filename=filename)
trial_metadata = pd.read_csv(filename)
####################################################################################################################
##### Load FC mappings
tools_group = reload(tools_group)
global fc_input2hidden
global fc_12rule2hidden
inputtypes = ['color','ori','pitch','constant']
inputkeys = ['RED','VERTICAL','HIGH','CONSTANT']
fc_input2hidden = {}
eig_input2hidden = {}
i = 0
for inputtype in inputtypes:
    fc_input2hidden[inputkeys[i]], eig_input2hidden[inputkeys[i]] = tools_group.loadGroupActFlowFC(inputtype, n_hiddenregions=n_hiddenregions, pc_space=True)
    print(fc_input2hidden[inputkeys[i]].shape)
    i += 1

fc_12rule2hidden, eig_12rule2hidden = tools_group.loadGroupActFlowFC('12', n_hiddenregions=n_hiddenregions, pc_space=True)
print(fc_12rule2hidden.shape)
# Load hidden to motor resp mappings
fc_hidden2motorresp, eig_hidden2motorresp = tools_group.loadGroupActFlowFC('hidden2out', n_hiddenregions=n_hiddenregions, pc_space=True)
print(fc_hidden2motorresp.shape)
####################################################################################################################

def subjSRActFlow_12Rule_PCFC(subj,trial_metadata,n_hiddenregions):
    print('Subject ' + subj + '... Simulating ' + str(len(trial_metadata)) + ' Trials')
    obj = esr.EmpiricalActFlow(subj)
    # Input
    obj.fc_input2hidden = fc_input2hidden
    obj.eig_input2hidden = eig_input2hidden
    # Rules
    obj.fc_12rule2hidden = fc_12rule2hidden
    obj.eig_12rule2hidden = eig_12rule2hidden
    # hidden 2 motor
    obj.fc_hidden2motorresp = fc_hidden2motorresp
    obj.eig_hidden2motorresp = eig_hidden2motorresp
    
    obj.extractAllActivations(trial_metadata)

    actflow = obj.generateActFlowPredictions_12Rule_PCFC(thresh=0,n_hiddenregions=n_hiddenregions,verbose=False)
    actflow_control = obj.generateActFlowPredictions_12Rule_PCFC(thresh=None,n_hiddenregions=n_hiddenregions,verbose=False)
    del obj
    return actflow, actflow_control

inputs = []
for i in range(len(subjNums)):
    inputs.append((subjNums[i],trial_metadata,n_hiddenregions))

timestart = time.time()
pool = mp.Pool(processes=8)
results = pool.starmap_async(subjSRActFlow_12Rule_PCFC,inputs).get()
pool.close()
pool.join()
timeend = time.time()
print("time elapsed:", timeend-timestart)

actflow_predictions = np.zeros((len(subjNums),len(target_ind),4))
actflow_predictions_control = np.zeros((len(subjNums),len(target_ind),4))
scount = 0
for result in results:
    actflow_predictions[scount,:,:] = result[0]
    actflow_predictions_control[scount,:,:] = result[1]
    scount += 1



(309, 905)
(500, 905)
(500, 905)
(500, 905)
(500, 905)
(500, 8642)
Subject 013... Simulating 960 Trials
Subject 017... Simulating 960 Trials
Subject 023... Simulating 960 Trials
Subject 027... Simulating 960 Trials
Subject 031... Simulating 960 Trials
Subject 038... Simulating 960 Trials
Subject 034... Simulating 960 Trials
Subject 041... Simulating 960 Trials
Subject 035... Simulating 960 Trials
Subject 042... Simulating 960 Trials
Subject 024... Simulating 960 Trials
Subject 028... Simulating 960 Trials
Subject 018... Simulating 960 Trials
Subject 039... Simulating 960 Trials
Subject 014... Simulating 960 Trials
Subject 032... Simulating 960 Trials
Subject 037... Simulating 960 Trials
Subject 043... Simulating 960 Trials
Subject 030... Simulating 960 Trials
Subject 026... Simulating 960 Trials
Subject 016... Simulating 960 Trials
Subject 021... Simulating 960 Trials
Subject 033... Simulating 960 Trials
Subject 040... Simulating 960 Trials
Subject 045... Simulating 960 Trials
Subject 

#### Compute average activity for each response, for each subject

In [52]:
## No threshold
scount = 0
actflow_rh = np.zeros(data_task_rh.shape)
actflow_lh = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh[target_ind,0,scount] = actflow_predictions[scount,:,2]
    # RIND
    actflow_rh[target_ind,1,scount] = actflow_predictions[scount,:,3]
    # LMID
    actflow_lh[target_ind,0,scount] = actflow_predictions[scount,:,0]
    # LIND
    actflow_lh[target_ind,1,scount] = actflow_predictions[scount,:,1]


## -10 HIDDEN - Run across subject decoding on right-hand motor responses

In [53]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputRH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]

# rois = np.asarray([7,8,52])
rois = np.asarray([8])
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind
#
realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh[roi_ind,:,:].copy()

#############################################################################################################
#### Run decoding
distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch, confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True, featsel=False,confusion=True,
                                                          ncvs=ncvs, nproc=nproc)

#############################################################################################################
#### Run statistics
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print('Activity flow accuracy =', statistics_rh[0,0])
print('p =', statistics_rh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))

Activity flow accuracy = 0.3125
p = 0.999999890906967
Matched spatial correlation: -0.03204070928334601
Mismatched spatial correlation: 0.03204070928334485


## -10 HIDDEN - Run across subject decoding on left-hand motor responses

In [54]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputLH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]

rois = np.asarray([187, 188, 231])
rois = np.asarray([188])
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind

realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh[roi_ind,:,:].copy()


#############################################################################################################
#### Run decoding
distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch, confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,featsel=False,confusion=True,
                                                          ncvs=ncvs, nproc=nproc)
    

#############################################################################################################
#### Run statistics
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print('Activity flow accuracy =', statistics_lh[0,0])
print('p =', statistics_lh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))

Activity flow accuracy = 0.65625
p = 8.903517788520931e-06
Matched spatial correlation: 0.0032771615199458804
Mismatched spatial correlation: -0.0032771615199458843


___

## Run analysis for Control (No threshold) variant

#### Compute average activity for each response, for each subject

In [55]:
scount = 0
actflow_rh_ctrl = np.zeros(data_task_rh.shape)
actflow_lh_ctrl = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh_ctrl[target_ind,0,scount] = actflow_predictions_control[scount,:,2]
    # RIND
    actflow_rh_ctrl[target_ind,1,scount] = actflow_predictions_control[scount,:,3]
    # LMID
    actflow_lh_ctrl[target_ind,0,scount] = actflow_predictions_control[scount,:,0]
    # LIND
    actflow_lh_ctrl[target_ind,1,scount] = actflow_predictions_control[scount,:,1]


## -10 HIDDEN - (NO THRESHOLD CONTROL): Run across subject decoding on right-hand motor responses

In [56]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputRH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]


# rois = np.asarray([8,52,9])-1
# rois = np.asarray([8,52])-1
# rois = np.where(networkdef==networkmappings['smn'])[0]
# rois = np.asarray([7,8,52])
rois = np.asarray([8])
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind

realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh_ctrl[roi_ind,:,:].copy()

#############################################################################################################
#### Run decoding
distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch, confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,confusion=True,featsel=False,
                                                          ncvs=ncvs, nproc=nproc)

#############################################################################################################
#### Run decoding
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print('Activity flow accuracy =', statistics_rh[0,0])
print('p =', statistics_rh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))



Activity flow accuracy = 0.4947916666666667
p = 0.528753715173387
Matched spatial correlation: 0.0026702673627045747
Mismatched spatial correlation: -0.00027649413034886995


## -10 HIDDEN - (NO THRESHOLD CONTROL): Run across subject decoding on left-hand motor responses

In [57]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputLH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]
# rois = np.asarray([188,189,232]) - 1
# rois = np.asarray([188,232]) - 1
# rois = np.asarray([189]) - 1
# rois = np.where(networkdef==networkmappings['smn'])[0]
# rois = np.asarray([187,188,231])
rois = np.asarray([188])
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind

realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh_ctrl[roi_ind,:,:].copy()


#############################################################################################################
#### Run decoding
distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch,confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,confusion=True,featsel=False,
                                                          ncvs=ncvs, nproc=nproc)
    

#############################################################################################################
#### Run statistics
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print('Activity flow accuracy =', statistics_lh[0,0])
print('p =', statistics_lh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))



Activity flow accuracy = 0.7239583333333334
p = 2.1940395714611462e-10
Matched spatial correlation: 0.01779711671878016
Mismatched spatial correlation: -0.014812824140803175


___

# -15 Hidden layers

#### Define wrapper function for SRActFlow

In [58]:
esr = reload(esr)
n_hiddenregions = -15
####################################################################################################################
#### Identify target vertices (in motor cortex)
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))

target_ind = []
for roi in targetROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)
####################################################################################################################
#### Generate trials to simulate empirical brain computational models
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_15stims_v2.csv' # Good
filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_15stims_v3.csv' # Great
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_6stims_v1.csv' # Great
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_50stims_v1.csv' # Great
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/tmp.csv'
# esr.constructTasks(n_stims=6,filename=filename)
trial_metadata = pd.read_csv(filename)
####################################################################################################################
##### Load FC mappings
tools_group = reload(tools_group)
global fc_input2hidden
global fc_12rule2hidden
inputtypes = ['color','ori','pitch','constant']
inputkeys = ['RED','VERTICAL','HIGH','CONSTANT']
fc_input2hidden = {}
eig_input2hidden = {}
i = 0
for inputtype in inputtypes:
    fc_input2hidden[inputkeys[i]], eig_input2hidden[inputkeys[i]] = tools_group.loadGroupActFlowFC(inputtype, n_hiddenregions=n_hiddenregions, pc_space=True)
    print(fc_input2hidden[inputkeys[i]].shape)
    i += 1

fc_12rule2hidden, eig_12rule2hidden = tools_group.loadGroupActFlowFC('12', n_hiddenregions=n_hiddenregions, pc_space=True)
print(fc_12rule2hidden.shape)
# Load hidden to motor resp mappings
fc_hidden2motorresp, eig_hidden2motorresp = tools_group.loadGroupActFlowFC('hidden2out', n_hiddenregions=n_hiddenregions, pc_space=True)
print(fc_hidden2motorresp.shape)
####################################################################################################################

def subjSRActFlow_12Rule_PCFC(subj,trial_metadata,n_hiddenregions):
    print('Subject ' + subj + '... Simulating ' + str(len(trial_metadata)) + ' Trials')
    obj = esr.EmpiricalActFlow(subj)
    # Input
    obj.fc_input2hidden = fc_input2hidden
    obj.eig_input2hidden = eig_input2hidden
    # Rules
    obj.fc_12rule2hidden = fc_12rule2hidden
    obj.eig_12rule2hidden = eig_12rule2hidden
    # hidden 2 motor
    obj.fc_hidden2motorresp = fc_hidden2motorresp
    obj.eig_hidden2motorresp = eig_hidden2motorresp
    
    obj.extractAllActivations(trial_metadata)

    actflow = obj.generateActFlowPredictions_12Rule_PCFC(thresh=0,n_hiddenregions=n_hiddenregions,verbose=False)
    actflow_control = obj.generateActFlowPredictions_12Rule_PCFC(thresh=None,n_hiddenregions=n_hiddenregions,verbose=False)
    del obj
    return actflow, actflow_control

inputs = []
for i in range(len(subjNums)):
    inputs.append((subjNums[i],trial_metadata,n_hiddenregions))

timestart = time.time()
pool = mp.Pool(processes=8)
results = pool.starmap_async(subjSRActFlow_12Rule_PCFC,inputs).get()
pool.close()
pool.join()
timeend = time.time()
print("time elapsed:", timeend-timestart)

actflow_predictions = np.zeros((len(subjNums),len(target_ind),4))
actflow_predictions_control = np.zeros((len(subjNums),len(target_ind),4))
scount = 0
for result in results:
    actflow_predictions[scount,:,:] = result[0]
    actflow_predictions_control[scount,:,:] = result[1]
    scount += 1



(190, 1306)
(500, 1306)
(500, 1306)
(500, 1306)
(500, 1306)
(500, 8642)
Subject 013... Simulating 960 Trials
Subject 017... Simulating 960 Trials
Subject 027... Simulating 960 Trials
Subject 023... Simulating 960 Trials
Subject 031... Simulating 960 Trials
Subject 034... Simulating 960 Trials
Subject 041... Simulating 960 Trials
Subject 038... Simulating 960 Trials
Subject 035... Simulating 960 Trials
Subject 018... Simulating 960 Trials
Subject 032... Simulating 960 Trials
Subject 039... Simulating 960 Trials
Subject 042... Simulating 960 Trials
Subject 028... Simulating 960 Trials
Subject 014... Simulating 960 Trials
Subject 024... Simulating 960 Trials
Subject 021... Simulating 960 Trials
Subject 037... Simulating 960 Trials
Subject 030... Simulating 960 Trials
Subject 040... Simulating 960 Trials
Subject 043... Simulating 960 Trials
Subject 016... Simulating 960 Trials
Subject 033... Simulating 960 Trials
Subject 026... Simulating 960 Trials
Subject 045... Simulating 960 Trials
Sub

#### Compute average activity for each response, for each subject

In [59]:
## No threshold
scount = 0
actflow_rh = np.zeros(data_task_rh.shape)
actflow_lh = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh[target_ind,0,scount] = actflow_predictions[scount,:,2]
    # RIND
    actflow_rh[target_ind,1,scount] = actflow_predictions[scount,:,3]
    # LMID
    actflow_lh[target_ind,0,scount] = actflow_predictions[scount,:,0]
    # LIND
    actflow_lh[target_ind,1,scount] = actflow_predictions[scount,:,1]


## -15 HIDDEN - Run across subject decoding on right-hand motor responses

In [60]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputRH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]

# rois = np.asarray([7,8,52])
# rois = np.asarray([8])
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind
#
realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh[roi_ind,:,:].copy()


#############################################################################################################
#### Run decoding
distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch, confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True, featsel=False,confusion=True,
                                                          ncvs=ncvs, nproc=nproc)

#############################################################################################################
#### Run statistics
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print('Activity flow accuracy =', statistics_rh[0,0])
print('p =', statistics_rh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))

Activity flow accuracy = 0.65625
p = 8.903517788520931e-06
Matched spatial correlation: 0.007148702695800976
Mismatched spatial correlation: -0.007148702695801072


## -15 HIDDEN - Run across subject decoding on left-hand motor responses

In [61]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputLH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]

# rois = np.asarray([187, 188, 231])
# rois = np.asarray([188])
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind

realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh[roi_ind,:,:].copy()


#############################################################################################################
#### Run decoding
distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch, confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,featsel=False,confusion=True,
                                                          ncvs=ncvs, nproc=nproc)
    

#############################################################################################################
#### Run statistics
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print('Activity flow accuracy =', statistics_lh[0,0])
print('p =', statistics_lh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))

Activity flow accuracy = 0.6354166666666666
p = 0.00010729988317582263
Matched spatial correlation: 0.018964837343907203
Mismatched spatial correlation: -0.018964837343911845


___

## Run analysis for Control (No threshold) variant

#### Compute average activity for each response, for each subject

In [62]:
scount = 0
actflow_rh_ctrl = np.zeros(data_task_rh.shape)
actflow_lh_ctrl = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh_ctrl[target_ind,0,scount] = actflow_predictions_control[scount,:,2]
    # RIND
    actflow_rh_ctrl[target_ind,1,scount] = actflow_predictions_control[scount,:,3]
    # LMID
    actflow_lh_ctrl[target_ind,0,scount] = actflow_predictions_control[scount,:,0]
    # LIND
    actflow_lh_ctrl[target_ind,1,scount] = actflow_predictions_control[scount,:,1]


## -15 HIDDEN - (NO THRESHOLD CONTROL): Run across subject decoding on right-hand motor responses

#### Run decoding

In [63]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputRH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]


# rois = np.asarray([8,52,9])-1
# rois = np.asarray([8,52])-1
# rois = np.where(networkdef==networkmappings['smn'])[0]
# rois = np.asarray([7,8,52])
# rois = np.asarray([8])
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind

realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh_ctrl[roi_ind,:,:].copy()

#############################################################################################################
#### Run decoding
distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch, confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,confusion=True,featsel=False,
                                                          ncvs=ncvs, nproc=nproc)

#############################################################################################################
#### Run statistics
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print('Activity flow accuracy =', statistics_rh[0,0])
print('p =', statistics_rh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))



Activity flow accuracy = 0.5
p = 0.5
Matched spatial correlation: -0.011976944168737583
Mismatched spatial correlation: -0.012162720526405846


## -15 HIDDEN - (NO THRESHOLD CONTROL): Run across subject decoding on left-hand motor responses

#### Run decoding

In [64]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputLH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]
# rois = np.asarray([188,189,232]) - 1
# rois = np.asarray([188,232]) - 1
# rois = np.asarray([189]) - 1
# rois = np.where(networkdef==networkmappings['smn'])[0]
# rois = np.asarray([187,188,231])
# rois = np.asarray([188])
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind

realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh_ctrl[roi_ind,:,:].copy()


#############################################################################################################
#### Run decoding
distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch,confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,confusion=True,featsel=False,
                                                          ncvs=ncvs, nproc=nproc)
    
#############################################################################################################
#### Run statistics
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print('Activity flow accuracy =', statistics_lh[0,0])
print('p =', statistics_lh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))



Activity flow accuracy = 0.484375
p = 0.6408405728911661
Matched spatial correlation: 0.013311596736603613
Mismatched spatial correlation: 0.01764634741256939


___

# -20 Hidden layers

#### Define wrapper function for SRActFlow

In [65]:
esr = reload(esr)
n_hiddenregions = -20
####################################################################################################################
#### Identify target vertices (in motor cortex)
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))

target_ind = []
for roi in targetROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)
####################################################################################################################
#### Generate trials to simulate empirical brain computational models
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_15stims_v2.csv' # Good
filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_15stims_v3.csv' # Great
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_6stims_v1.csv' # Great
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/EmpiricalSRActFlow_AllTrialKeys_50stims_v1.csv' # Great
# filename='/projects3/SRActFlow/data/results/GroupfMRI/RSA/tmp.csv'
# esr.constructTasks(n_stims=6,filename=filename)
trial_metadata = pd.read_csv(filename)
####################################################################################################################
##### Load FC mappings
tools_group = reload(tools_group)
global fc_input2hidden
global fc_12rule2hidden
inputtypes = ['color','ori','pitch','constant']
inputkeys = ['RED','VERTICAL','HIGH','CONSTANT']
fc_input2hidden = {}
eig_input2hidden = {}
i = 0
for inputtype in inputtypes:
    fc_input2hidden[inputkeys[i]], eig_input2hidden[inputkeys[i]] = tools_group.loadGroupActFlowFC(inputtype, n_hiddenregions=n_hiddenregions, pc_space=True)
    print(fc_input2hidden[inputkeys[i]].shape)
    i += 1

fc_12rule2hidden, eig_12rule2hidden = tools_group.loadGroupActFlowFC('12', n_hiddenregions=n_hiddenregions, pc_space=True)
print(fc_12rule2hidden.shape)
# Load hidden to motor resp mappings
fc_hidden2motorresp, eig_hidden2motorresp = tools_group.loadGroupActFlowFC('hidden2out', n_hiddenregions=n_hiddenregions, pc_space=True)
print(fc_hidden2motorresp.shape)
####################################################################################################################


def subjSRActFlow_12Rule_PCFC(subj,trial_metadata,n_hiddenregions):
    print('Subject ' + subj + '... Simulating ' + str(len(trial_metadata)) + ' Trials')
    obj = esr.EmpiricalActFlow(subj)
    # Input
    obj.fc_input2hidden = fc_input2hidden
    obj.eig_input2hidden = eig_input2hidden
    # Rules
    obj.fc_12rule2hidden = fc_12rule2hidden
    obj.eig_12rule2hidden = eig_12rule2hidden
    # hidden 2 motor
    obj.fc_hidden2motorresp = fc_hidden2motorresp
    obj.eig_hidden2motorresp = eig_hidden2motorresp
    
    obj.extractAllActivations(trial_metadata)

    actflow = obj.generateActFlowPredictions_12Rule_PCFC(thresh=0,n_hiddenregions=n_hiddenregions,verbose=False)
    actflow_control = obj.generateActFlowPredictions_12Rule_PCFC(thresh=None,n_hiddenregions=n_hiddenregions,verbose=False)
    del obj
    return actflow, actflow_control

inputs = []
for i in range(len(subjNums)):
    inputs.append((subjNums[i],trial_metadata,n_hiddenregions))

timestart = time.time()
pool = mp.Pool(processes=8)
results = pool.starmap_async(subjSRActFlow_12Rule_PCFC,inputs).get()
pool.close()
pool.join()
timeend = time.time()
print("time elapsed:", timeend-timestart)

actflow_predictions = np.zeros((len(subjNums),len(target_ind),4))
actflow_predictions_control = np.zeros((len(subjNums),len(target_ind),4))
scount = 0
for result in results:
    actflow_predictions[scount,:,:] = result[0]
    actflow_predictions_control[scount,:,:] = result[1]
    scount += 1



(190, 1710)
(500, 1710)
(500, 1710)
(500, 1710)
(500, 1710)
(500, 8642)
Subject 013... Simulating 960 Trials
Subject 023... Simulating 960 Trials
Subject 017... Simulating 960 Trials
Subject 027... Simulating 960 Trials
Subject 031... Simulating 960 Trials
Subject 034... Simulating 960 Trials
Subject 038... Simulating 960 Trials
Subject 041... Simulating 960 Trials
Subject 014... Simulating 960 Trials
Subject 042... Simulating 960 Trials
Subject 024... Simulating 960 Trials
Subject 018... Simulating 960 Trials
Subject 039... Simulating 960 Trials
Subject 028... Simulating 960 Trials
Subject 035... Simulating 960 Trials
Subject 032... Simulating 960 Trials
Subject 016... Simulating 960 Trials
Subject 043... Simulating 960 Trials
Subject 021... Simulating 960 Trials
Subject 026... Simulating 960 Trials
Subject 040... Simulating 960 Trials
Subject 030... Simulating 960 Trials
Subject 033... Simulating 960 Trials
Subject 037... Simulating 960 Trials
Subject 045... Simulating 960 Trials
Sub

#### Compute average activity for each response, for each subject

In [66]:
## No threshold
scount = 0
actflow_rh = np.zeros(data_task_rh.shape)
actflow_lh = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh[target_ind,0,scount] = actflow_predictions[scount,:,2]
    # RIND
    actflow_rh[target_ind,1,scount] = actflow_predictions[scount,:,3]
    # LMID
    actflow_lh[target_ind,0,scount] = actflow_predictions[scount,:,0]
    # LIND
    actflow_lh[target_ind,1,scount] = actflow_predictions[scount,:,1]


## -20 HIDDEN - Run across subject decoding on right-hand motor responses

In [67]:
#### tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputRH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]

# rois = np.asarray([7,8,52])
# rois = np.asarray([8])
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind
#
realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh[roi_ind,:,:].copy()


#############################################################################################################
#### Run decoding
distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch, confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True, featsel=False,confusion=True,
                                                          ncvs=ncvs, nproc=nproc)

#############################################################################################################
#### Run statistics
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print('Activity flow accuracy =', statistics_rh[0,0])
print('p =', statistics_rh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))

Activity flow accuracy = 0.375
p = 0.9996724848250989
Matched spatial correlation: -0.02792278842492062
Mismatched spatial correlation: 0.02792278842491588


## -20 HIDDEN - Run across subject decoding on left-hand motor responses

In [68]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputLH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]

# rois = np.asarray([187, 188, 231])
# rois = np.asarray([188])
# rois = np.where(networkdef==networkmappings['smn'])[0]
# roi_ind = []
# for roi in rois:
#     roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind

realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh[roi_ind,:,:].copy()


#############################################################################################################
#### Run decoding
distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch, confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,featsel=False,confusion=True,
                                                          ncvs=ncvs, nproc=nproc)
    

#############################################################################################################
#### Run statistics
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print('Activity flow accuracy =', statistics_lh[0,0])
print('p =', statistics_lh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))

Activity flow accuracy = 0.40625
p = 0.9943272543485387
Matched spatial correlation: -0.006922789554472286
Mismatched spatial correlation: 0.006922789554471782


___

## Run analysis for Control (No threshold) variant

#### Compute average activity for each response, for each subject

In [69]:
scount = 0
actflow_rh_ctrl = np.zeros(data_task_rh.shape)
actflow_lh_ctrl = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh_ctrl[target_ind,0,scount] = actflow_predictions_control[scount,:,2]
    # RIND
    actflow_rh_ctrl[target_ind,1,scount] = actflow_predictions_control[scount,:,3]
    # LMID
    actflow_lh_ctrl[target_ind,0,scount] = actflow_predictions_control[scount,:,0]
    # LIND
    actflow_lh_ctrl[target_ind,1,scount] = actflow_predictions_control[scount,:,1]


## -20 HIDDEN - (NO THRESHOLD CONTROL): Run across subject decoding on right-hand motor responses

In [70]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputRH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]


# rois = np.asarray([8,52,9])-1
# rois = np.asarray([8,52])-1
# rois = np.where(networkdef==networkmappings['smn'])[0]
# rois = np.asarray([7,8,52])
rois = np.asarray([8])
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
# roi_ind = targetmask_ind

realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh_ctrl[roi_ind,:,:].copy()


#############################################################################################################
#### Run decoding
distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch, confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,confusion=True,featsel=False,
                                                          ncvs=ncvs, nproc=nproc)

#############################################################################################################
#### Run statistics
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print('Activity flow accuracy =', statistics_rh[0,0])
print('p =', statistics_rh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))

Activity flow accuracy = 0.46875
p = 0.7863227053016961
Matched spatial correlation: 0.0003421814067378775
Mismatched spatial correlation: 0.0018952629097658627


## -20 HIDDEN - (NO THRESHOLD CONTROL): Run across subject decoding on left-hand motor responses

In [71]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

tmp = np.squeeze(nib.load('/projects3/SRActFlow/data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputLH_mask.dscalar.nii').get_data())

targetmask_ind = np.where(tmp==True)[0]
# rois = np.asarray([188,189,232]) - 1
# rois = np.asarray([188,232]) - 1
# rois = np.asarray([189]) - 1
# rois = np.where(networkdef==networkmappings['smn'])[0]
# rois = np.asarray([187,188,231])
rois = np.asarray([188])
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# roi_ind = np.intersect1d(targetmask_ind,roi_ind)
roi_ind = targetmask_ind

realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh_ctrl[roi_ind,:,:].copy()


#############################################################################################################
#### Run decoding
distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch,confusion_mats = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,confusion=True,featsel=False,
                                                          ncvs=ncvs, nproc=nproc)
    
#############################################################################################################
#### Run statistics
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print('Activity flow accuracy =', statistics_lh[0,0])
print('p =', statistics_lh[0,1])
print('Matched spatial correlation:', np.mean(rmatch))
print('Mismatched spatial correlation:', np.mean(rmismatch))

Activity flow accuracy = 0.5364583333333334
p = 0.17408558356591572
Matched spatial correlation: 0.002279313391171287
Mismatched spatial correlation: -0.006582102007937045


___